##Data Processing





In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer

In [7]:
data = pd.read_csv("hotel_bookings.csv")

In [8]:
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

###Encoding

In [10]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

In [11]:
data['hotel'].value_counts()

City Hotel      79330
Resort Hotel    40060
Name: hotel, dtype: int64

###Label Encoder

In [12]:
le = LabelEncoder()
data['hotel'] = le.fit_transform(data['hotel'])

In [13]:
data['hotel'].value_counts()

0    79330
1    40060
Name: hotel, dtype: int64

In [14]:
le.classes_

array(['City Hotel', 'Resort Hotel'], dtype=object)

###One Hot Encoder

In [15]:
data['customer_type'].value_counts()

Transient          89613
Transient-Party    25124
Contract            4076
Group                577
Name: customer_type, dtype: int64

In [16]:
one_hot = OneHotEncoder()
transformed_data = one_hot.fit_transform(data['customer_type'].values.reshape(-1,1)).toarray()

one_hot.categories_

[array(['Contract', 'Group', 'Transient', 'Transient-Party'], dtype=object)]

In [17]:
transformed_data = pd.DataFrame(transformed_data , 
                                columns = ['Contract', 'Group', 'Transient', 'Transient-Party'])

transformed_data.head()

,Contract,Group,Transient,Transient-Party
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [18]:
transformed_data.iloc[90 , ]

Contract           0.0
Group              0.0
Transient          1.0
Transient-Party    0.0
Name: 90, dtype: float64

In [19]:
data['customer_type'][90]

'Transient'

###Normalization and Standardization

In [20]:
numeric_columns = [c for c in data.columns if data[c].dtype != np.dtype('O')]

len(numeric_columns) , len(data.columns)

(21, 32)

In [21]:
numeric_columns.remove('company')
numeric_columns.remove('agent')

temp_data = data[numeric_columns]

temp_data

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1,0,342,2015,27,1,0,0,2,0.0,0,0,0,0,3,0,0.00,0,0
1,1,0,737,2015,27,1,0,0,2,0.0,0,0,0,0,4,0,0.00,0,0
2,1,0,7,2015,27,1,0,1,1,0.0,0,0,0,0,0,0,75.00,0,0
3,1,0,13,2015,27,1,0,1,1,0.0,0,0,0,0,0,0,75.00,0,0
4,1,0,14,2015,27,1,0,2,2,0.0,0,0,0,0,0,0,98.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,0,23,2017,35,30,2,5,2,0.0,0,0,0,0,0,0,96.14,0,0
119386,0,0,102,2017,35,31,2,5,3,0.0,0,0,0,0,0,0,225.43,0,2
119387,0,0,34,2017,35,31,2,5,2,0.0,0,0,0,0,0,0,157.71,0,4
119388,0,0,109,2017,35,31,2,5,2,0.0,0,0,0,0,0,0,104.40,0,0


###Normalization

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
normalizer = MinMaxScaler()

temp_data.dropna(axis = 1 , inplace = True)

normalized_data = normalizer.fit_transform(temp_data)

pd.DataFrame(normalized_data , columns = temp_data.columns)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1.0,0.0,0.464043,0.0,0.500000,0.000000,0.000000,0.00,0.036364,0.0,0.0,0.0,0.0,0.142857,0.0,0.001180,0.0,0.0
1,1.0,0.0,1.000000,0.0,0.500000,0.000000,0.000000,0.00,0.036364,0.0,0.0,0.0,0.0,0.190476,0.0,0.001180,0.0,0.0
2,1.0,0.0,0.009498,0.0,0.500000,0.000000,0.000000,0.02,0.018182,0.0,0.0,0.0,0.0,0.000000,0.0,0.015053,0.0,0.0
3,1.0,0.0,0.017639,0.0,0.500000,0.000000,0.000000,0.02,0.018182,0.0,0.0,0.0,0.0,0.000000,0.0,0.015053,0.0,0.0
4,1.0,0.0,0.018996,0.0,0.500000,0.000000,0.000000,0.04,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.019307,0.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.0,0.0,0.031208,1.0,0.653846,0.966667,0.105263,0.10,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.018963,0.0,0.0
119386,0.0,0.0,0.138399,1.0,0.653846,1.000000,0.105263,0.10,0.054545,0.0,0.0,0.0,0.0,0.000000,0.0,0.042877,0.0,0.4
119387,0.0,0.0,0.046133,1.0,0.653846,1.000000,0.105263,0.10,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.030351,0.0,0.8
119388,0.0,0.0,0.147897,1.0,0.653846,1.000000,0.105263,0.10,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.020491,0.0,0.0


###Standardization

In [24]:
standard_scaler = StandardScaler()

standardized_data = standard_scaler.fit_transform(temp_data)

pd.DataFrame(standardized_data , columns = temp_data.columns)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1.407224,-0.76704,2.227051,-1.634768,-0.012141,-1.685297,-0.928890,-1.310240,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,4.260101,-0.131924,-2.015038,-0.254873,-0.720694
1,1.407224,-0.76704,5.923385,-1.634768,-0.012141,-1.685297,-0.928890,-1.310240,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,5.793131,-0.131924,-2.015038,-0.254873,-0.720694
2,1.407224,-0.76704,-0.907814,-1.634768,-0.012141,-1.685297,-0.928890,-0.786207,-1.478447,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.530935,-0.254873,-0.720694
3,1.407224,-0.76704,-0.851667,-1.634768,-0.012141,-1.685297,-0.928890,-0.786207,-1.478447,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.530935,-0.254873,-0.720694
4,1.407224,-0.76704,-0.842309,-1.634768,-0.012141,-1.685297,-0.928890,-0.262174,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.075810,-0.254873,0.540666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,-0.710619,-0.76704,-0.758089,1.192195,0.575875,1.617366,1.073895,1.309924,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.112616,-0.254873,-0.720694
119386,-0.710619,-0.76704,-0.018822,1.192195,0.575875,1.731251,1.073895,1.309924,1.974242,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,2.445779,-0.254873,1.802026
119387,-0.710619,-0.76704,-0.655153,1.192195,0.575875,1.731251,1.073895,1.309924,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,1.105733,-0.254873,4.324746
119388,-0.710619,-0.76704,0.046682,1.192195,0.575875,1.731251,1.073895,1.309924,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,0.050833,-0.254873,-0.720694


###Missing values

In [25]:
data.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [26]:
data['agent'].isnull().sum()

16340

###Simple Impuler

In [27]:
imputer = SimpleImputer(missing_values=np.nan , strategy='mean')
agent_col = imputer.fit_transform(data['agent'].values.reshape(-1,1))
pd.DataFrame(agent_col).isnull().sum()

0    0
dtype: int64

In [28]:
data['agent'].isnull().sum()

16340

###Discretization

In [29]:
temp_data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1,0,342,2015,27,1,0,0,2,0,0,0,0,3,0,0.0,0,0
1,1,0,737,2015,27,1,0,0,2,0,0,0,0,4,0,0.0,0,0
2,1,0,7,2015,27,1,0,1,1,0,0,0,0,0,0,75.0,0,0
3,1,0,13,2015,27,1,0,1,1,0,0,0,0,0,0,75.0,0,0
4,1,0,14,2015,27,1,0,2,2,0,0,0,0,0,0,98.0,0,1


###Quantile Discretization Transform

In [30]:
trans = KBinsDiscretizer(n_bins =10 , encode = 'ordinal' , strategy='quantile')
new_data = trans.fit_transform(temp_data)

In [31]:
pd.DataFrame(new_data,columns = temp_data.columns )

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,0.0,0.0,9.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,9.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3,0.0,0.0,2.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,0.0,0.0,2.0,0.0,4.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.0,0.0,2.0,1.0,6.0,9.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
119386,0.0,0.0,6.0,1.0,6.0,9.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,2.0
119387,0.0,0.0,3.0,1.0,6.0,9.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,2.0
119388,0.0,0.0,6.0,1.0,6.0,9.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


###Uniform Discretization Transfom

In [32]:
trans = KBinsDiscretizer(n_bins =10 , encode = 'ordinal' , strategy='uniform')
new_data = trans.fit_transform(temp_data)

pd.DataFrame(new_data,columns = temp_data.columns )

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,9.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,9.0,0.0,9.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,9.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.0,0.0,0.0,9.0,6.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119386,0.0,0.0,1.0,9.0,6.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
119387,0.0,0.0,0.0,9.0,6.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
119388,0.0,0.0,1.0,9.0,6.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


###Kmeans Discretization Transform

In [33]:
trans = KBinsDiscretizer(n_bins =10 , encode = 'ordinal' , strategy='kmeans')
new_data = trans.fit_transform(temp_data)

pd.DataFrame(new_data,columns = temp_data.columns )

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,9.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3,1.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,1.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.0,0.0,0.0,2.0,6.0,9.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
119386,0.0,0.0,2.0,2.0,6.0,9.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,3.0
119387,0.0,0.0,1.0,2.0,6.0,9.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0
119388,0.0,0.0,2.0,2.0,6.0,9.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
